# Model description

**Pose landmarker model**

The MediaPipe Pose Landmarker task lets you detect landmarks of human bodies in an image or video. You can use this task to identify key body locations, analyze posture, and categorize movements. This task uses machine learning (ML) models that work with single images or video. The task outputs body pose landmarks in image coordinates and in 3-dimensional world coordinates.

The pose landmarker model tracks 33 body landmark locations, representing the approximate location of the following body parts:

0 - nose
1 - left eye (inner)
2 - left eye
3 - left eye (outer)
4 - right eye (inner)
5 - right eye
6 - right eye (outer)
7 - left ear
8 - right ear
9 - mouth (left)
10 - mouth (right)
11 - left shoulder
12 - right shoulder
13 - left elbow
14 - right elbow
15 - left wrist
16 - right wrist
17 - left pinky
18 - right pinky
19 - left index
20 - right index
21 - left thumb
22 - right thumb
23 - left hip
24 - right hip
25 - left knee
26 - right knee
27 - left ankle
28 - right ankle
29 - left heel
30 - right heel
31 - left foot index
32 - right foot index

<img src="https://developers.google.com/static/mediapipe/images/solutions/pose_landmarks_index.png" alt="Image" width="400">

By calculating the angle between three key points (shoulder - elbow - wrist) a curl excercise reps counter can be implemented.

# Initialization

In [ ]:
# !pip install opencv-python mediapipe

In [ ]:
# Load libraries
import cv2
import mediapipe as mp
import numpy as np
import contextlib

In [ ]:
# Load MediaPipe pose model
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
# Create curl counter class
class CurlCounter():
    def __init__(self, min_angle=30, max_angle=160, resting_angle=180):
        self.min_angle = min_angle
        self.max_angle = max_angle
        self.resting_angle = resting_angle
        self.current_angle = resting_angle
        self.counter = 0
        self.state = "down"
        self.landmarks = None

    def calculate_angle_3p(self, a, b, c):
        # Calculating the vectors ab and bc
        ab = np.array([a[0] - b[0], a[1] - b[1]])
        bc = np.array([c[0] - b[0], c[1] - b[1]])

        # Calculating the cosine of the angle using the dot product
        cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc))
        
        # Ensuring the cosine value is within the valid range [-1, 1]
        cosine_angle = np.clip(cosine_angle, -1.0, 1.0)

        # Calculating the angle in radians and converting it to degrees
        angle = np.arccos(cosine_angle)
        return np.degrees(angle)
    
    def is_visible(self):
        return self.landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility > 0.5

    def calc_angle(self):
        if self.is_visible():
            LEFT_SHOULDER = [self.landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, self.landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            LEFT_ELBOW = [self.landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, self.landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            LEFT_WRIST = [self.landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, self.landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            self.current_angle = self.calculate_angle_3p(LEFT_SHOULDER, LEFT_ELBOW, LEFT_WRIST)
        else:
            self.current_angle = self.resting_angle

    def count(self):
        if self.current_angle > self.max_angle and self.state == 'up':
            self.counter += 1
            self.state = 'down'
        if self.current_angle < self.min_angle and self.state == 'down':
            self.state = 'up'


In [ ]:
# Setup MediaPipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    # Read feed
    cap = cv2.VideoCapture(0)

    # Get screen dimensions
    screen_width = int(cap.get(3))
    screen_height = int(cap.get(4))

    # Create curl counter instance
    curl_counter = CurlCounter()

    while cap.isOpened():
        _, image = cap.read()

        # Flip image horizontally
        image = cv2.flip(image, 1)

        # Recolor feed
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor image back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        with contextlib.suppress(Exception):
            landmarks = results.pose_landmarks.landmark
        
            # curl counter
            curl_counter.landmarks = landmarks
            curl_counter.calc_angle()
            curl_counter.count()

            # Render curl counter
            cv2.putText(image, str(curl_counter.counter),
                        (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            
            # Render angle on left elbow
            if curl_counter.is_visible():
                cv2.putText(image, str(int(curl_counter.current_angle)),
                            tuple(np.multiply(
                                (landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y),
                                [screen_width, screen_height]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
        )

        # Show detections
        cv2.imshow('Detection Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
exit()